In [2]:
import concurrent.futures
import requests
from datetime import datetime, timedelta
import pandas as pd

pays_coordinee = {
    "senegal": {
        "dakar": (14.6928, -17.4467),
        "thies": (14.7894, -16.926),
        "saint-louis": (16.0333, -16.5),
        "kaolack": (14.151, -16.0726),
        "ziguinchor": (12.5833, -16.2667),
        "tambacounda": (13.77, -13.6672),
        "kedougou": (12.5535, -12.1743),
    },
    "mali": {
        "bamako": (12.6392, -8.0029),
        "segou": (13.4317, -6.2157),
        "timbuktu": (16.7666, -3.0026),
        "mopti": (14.4843, -4.1827),
    },
    "cote_d_ivoire": {
        "abidjan": (5.3364, -4.0261),
        "bouake": (7.6833, -5.0333),
        "yamoussoukro": (6.8161, -5.2742),
        "san_pedro": (4.7485, -6.6363),
    },
    "guinee": {
        "conakry": (9.6412, -13.5784),
        "kankan": (10.3842, -9.3057),
        "n_zerekore": (7.7594, -8.8174),
        "labé": (11.3167, -12.2833),
    },
    "nigeria": {
        "lagos": (6.5244, 3.3792),
        "abuja": (9.0579, 7.4951),
        "kano": (12.0022, 8.5919),
    },
    "ghana": {
        "accra": (5.6037, -0.187),
        "kumasi": (6.6666, -1.6163),
        "tamale": (9.4075, -0.8531),
        "takoradi": (4.8975, -1.7603),
    },
    "burkina faso": {
        "ouagadougou": (12.3714, -1.5197),
        "bobo dioulasso": (11.1786, -4.2979),
        "koudougou": (12.2542, -2.3625),
    },
}

def __get_url__(lat:str, long:str, start_date:str, end_date:str)->str:
    return f"https://power.larc.nasa.gov/api/temporal/hourly/point?parameters=T2M,RH2M,T2MWET,PRECTOT,WS10M,WD10M,T2MDEW,V10M,PS,QV2M,U10M&community=AG&longitude={long}&latitude={lat}&start={start_date}&end={end_date}&format=json"


def __convert_to_df_optimized__(parameters, city, county):
    # Créer directement un dictionnaire avec toutes les dates
    dates = list(parameters['T2M'].keys())
    n_dates = len(dates)
    
    data = {
        'date': dates,
        'ville': [city] * n_dates,
        'pays': [county] * n_dates
    }
    
    # Ajouter les paramètres en une seule étape
    for param in parameters:
        data[param] = [parameters[param].get(date, None) for date in dates]
    
    return pd.DataFrame(data)


def get_data(pays_list, start_date, end_date):
    dfs = []
    
    def fetch_city_data(pays, ville, coordonate):
        lat, long = coordonate
        url = __get_url__(lat=lat, long=long, start_date=start_date, end_date=end_date)
        try:
            response = requests.get(url)
            data = response.json()["properties"]["parameter"]
            return __convert_to_df_optimized__(parameters=data, city=ville, county=pays)
        except Exception as e:
            print(f"Erreur pour {ville}, {pays}: {e}")
            return None
    
    tasks = []
    for pays_loop in pays_list:
        if pays_loop.lower() not in [p.lower() for p in pays_coordinee.keys()]:
            print(f"Ce pays n'est pas pris en compte: {pays_loop}")
            continue
        
        villes = pays_coordinee[pays_loop.lower()]
        for ville, coordonate in villes.items():
            tasks.append((pays_loop, ville, coordonate))
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(fetch_city_data, pays, ville, coord) for pays, ville, coord in tasks]
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result is not None:
                dfs.append(result)
    
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()


date_actuelle = datetime.today()
date_delay = datetime.today() - timedelta(days=8)
end_date = date_delay.strftime("%Y%m%d")

# Date un mois avant
start_date_delay = date_delay - timedelta(days=75)
start_date = start_date_delay.strftime("%Y%m%d")

df = get_data(["Senegal","mali","cote_d_ivoire","guinee","nigeria","ghana","burkina faso"],start_date,end_date)
df.to_csv("Nasa_Power_data.csv", index=False)


Erreur pour labé, guinee: 'properties'
Erreur pour tamale, ghana: 'properties'


In [19]:
df = pd.read_csv("Nasa_Power_data.csv")
df.head()

,date,ville,pays,WS10M,QV2M,U10M,T2MWET,PS,T2M,WD10M,RH2M,V10M,T2MDEW,PRECTOTCORR
0,2025011500,saint-louis,Senegal,4.18,6.16,-1.45,12.01,101.01,17.46,20.3,49.94,-3.92,6.56,0.0
1,2025011501,saint-louis,Senegal,4.15,5.69,-1.68,11.21,100.95,16.92,23.9,47.72,-3.80,5.50,0.0
2,2025011502,saint-louis,Senegal,4.14,5.28,-1.82,10.43,100.92,16.36,26.1,45.88,-3.72,4.50,0.0
3,2025011503,saint-louis,Senegal,4.13,5.07,-1.86,9.85,100.91,15.74,26.8,45.84,-3.69,3.95,0.0
4,2025011504,saint-louis,Senegal,4.07,5.03,-1.75,9.48,100.95,15.11,25.5,47.37,-3.67,3.85,0.0


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, substring, to_date, concat_ws, to_timestamp, lit
import pandas as pd
spark = SparkSession.builder \
    .appName("Météo") \
    .config("spark.python.worker.reuse", "true") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.network.timeout", "600s") \
    .getOrCreate()

df_read = pd.read_csv("Nasa_Power_data.csv")

df = spark.createDataFrame(pd.DataFrame(df_read))

df = df.withColumn("date_str", col("date").cast("string"))
df = df.withColumn("date_formatted", to_date(substring(col("date_str"), 1, 8), "yyyyMMdd")) \
           .withColumn("heure_str", concat_ws(":", substring(col("date_str"), 9, 2), lit("00"), lit("00"))) \
           .withColumn("heure_formatted", concat_ws(" ", col("date_formatted"), col("heure_str")))
# Afficher le résultat
df.show(5)
spark.stop()

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, substring, to_date, concat_ws, to_timestamp, lit,date_format
import pandas as pd

spark = SparkSession.builder \
    .appName("Météo") \
    .config("spark.python.worker.reuse", "true") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.network.timeout", "600s") \
    .getOrCreate()

df_read = pd.read_csv("Nasa_Power_data.csv")
print(df_read.shape)

def date_hour_colonne(dataframe):
    # Créer le DataFrame Spark depuis Pandas
    df = spark.createDataFrame(pd.DataFrame(dataframe))
    
    # Sauvegarder la date brute dans une nouvelle colonne temporaire
    df = df.withColumn("date_str", col("date").cast("string"))
    
    # Extraire la date et l'heure
    df = df.withColumn("date_formatted", to_date(substring(col("date_str"), 1, 8), "yyyyMMdd")) \
           .withColumn("heure_formatted", concat_ws(":", substring(col("date_str"), 9, 2), lit("00"), lit("00"))) \
           #.withColumn("heure_formatted", concat_ws(" ", col("date_formatted"), col("heure_str"))) \
    
    # Obtenir une liste de colonnes sans les colonnes temporaires et les colonnes à remplacer
    all_columns = [c for c in df.columns if c not in ["date_str", "heure_str", "date", "heure", "date_formatted", "heure_formatted"]]
    
    # Sélectionner les colonnes originales plus les nouvelles colonnes transformées
    df = df.select(
        *all_columns,
        col("date_formatted").alias("date"),
        col("heure_formatted").alias("heure"),
    )
    
    return df

def nan_value_manage(df):
    return df.dropna()

def duplicate_value_manage(df):
    return df.dropDuplicates()

def remove_invalid_rows(df):
    # Supprimer les lignes où T2MWET < -30 ou RH2M < -30
    df = df.filter((col("T2MWET") >= -30) & (col("RH2M") >= -30))
    return df

def transformer_header(df):
    # Dictionnaire de correspondance entre les anciennes colonnes et les nouvelles colonnes
    header_map = {
        'ville': 'ville',
        'pays': 'pays',
        'T2M': 'temperature_air',
        'PS': 'pression',
        'WS10M': 'intensite_vent',
        'QV2M': 'humidite_specifique',
        'T2MDEW': 'temperature_point_rosee',
        'U10M': 'composante_est_ouest_vent',
        'V10M': 'vitesse_vent',
        'RH2M': 'humidite_relative',
        'WD10M': 'direction_vent',
        'T2MWET': 'temperature_humide',
        'PRECTOTCORR': 'precipitations_corrigees',
        'date': 'date',
        'heure': 'heure'
    }
    
    # Renommer les colonnes du DataFrame
    df.rename(columns=header_map, inplace=True)
    
    return df

(49248, 14)


In [21]:
df = date_hour_colonne(df_read)
# nan_value_manage
df = nan_value_manage(df)
# nan_value_manage
df = duplicate_value_manage(df)
df_cleaned = remove_invalid_rows(df)

df_cleaned = df.toPandas()

#Transforme header
df_cleaned = transformer_header(df_cleaned)


df_cleaned.shape

(49248, 15)

In [6]:
spark.stop()

In [7]:
df_cleaned.head()

,ville,pays,intensite_vent,humidite_specifique,composante_est_ouest_vent,temperature_humide,pression,temperature_air,direction_vent,humidite_relative,vitesse_vent,temperature_point_rosee,precipitations_corrigees,date,heure
0,saint-louis,Senegal,4.07,5.03,-1.75,9.48,100.95,15.11,25.5,47.37,-3.67,3.85,0.0,2025-01-15,04:00:00
1,saint-louis,Senegal,5.48,4.63,-3.00,10.74,101.19,18.74,33.2,34.73,-4.58,2.74,0.0,2025-01-15,08:00:00
2,saint-louis,Senegal,5.08,7.80,0.89,14.96,101.12,19.79,349.9,54.65,-5.00,10.13,0.0,2025-01-15,21:00:00
3,saint-louis,Senegal,4.80,8.60,0.48,15.23,101.11,18.81,354.3,64.02,-4.78,11.64,0.0,2025-01-15,22:00:00
4,saint-louis,Senegal,4.01,9.52,-1.20,14.01,101.09,15.56,17.4,86.99,-3.83,12.46,0.0,2025-01-16,04:00:00


In [22]:
import psycopg2

POSTGRES_USER = "postgres"
POSTGRES_PASSWORD = "passer"
POSTGRES_HOST = "localhost"
DB_NAME = "meteo_db"
TABLE_NAME = "meteo_data"

def __create_postgrest_database_if_not_exist__():
    try:
        # Connexion à la base de données 'postgres' pour vérifier si 'meteo_db' existe
        conn = psycopg2.connect(dbname="postgres", user=POSTGRES_USER, password=POSTGRES_PASSWORD, host=POSTGRES_HOST)
        conn.autocommit = True
        cur = conn.cursor()

        # Vérification si la base de données 'meteo_db' existe
        cur.execute(f"SELECT 1 FROM pg_database WHERE datname = '{DB_NAME}';")
        exists = cur.fetchone()
        if not exists:
            # Si la base de données n'existe pas, on la crée
            print(f"Creating database '{DB_NAME}'...")
            cur.execute(f"CREATE DATABASE {DB_NAME};")
            print(f"Database '{DB_NAME}' created successfully")
        else:
            print(f"Database '{DB_NAME}' already exists")
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Error creating database: {e}")
        return False

__create_postgrest_database_if_not_exist__()

Database 'meteo_db' already exists


In [23]:
def __create_cube_if_not_exist__():
    db_name = "meteo_cube"
    try:
        # Connexion à la base 'postgres' pour vérifier si la base cible existe
        conn = psycopg2.connect(
            dbname="postgres",
            user=POSTGRES_USER,
            password=POSTGRES_PASSWORD,
            host=POSTGRES_HOST
        )
        conn.autocommit = True  # ✅ Placer immédiatement après la connexion
        with conn.cursor() as cur:
            cur.execute("SELECT 1 FROM pg_database WHERE datname = %s;", (db_name,))
            exists = cur.fetchone()

            if not exists:
                print(f"Creating database '{db_name}'...")
                cur.execute(f"CREATE DATABASE {db_name};")
                print(f"Database '{db_name}' created successfully")
            else:
                print(f"Database '{db_name}' already exists")
        conn.close()

        # Connexion à la base de données cible pour créer les tables
        with psycopg2.connect(
            dbname=db_name,
            user=POSTGRES_USER,
            password=POSTGRES_PASSWORD,
            host=POSTGRES_HOST
        ) as conn:
            with conn.cursor() as cur:
                #cur.execute(f"CREATE SCHEMA IF NOT EXISTS {db_name};")

                cur.execute(f"""
                    CREATE TABLE IF NOT EXISTS dim_temps (
                        id SERIAL PRIMARY KEY,
                        date_key INTEGER NOT NULL,
                        heure TIME,
                        annee INT,
                        mois INT,
                        nom_mois VARCHAR(15),
                        jours INT,
                        jours_semaine INT,
                        nom_jours VARCHAR (255)
                    );
                """)
                cur.execute(f"""
                    CREATE TABLE IF NOT EXISTS dim_temperature_humide (
                        id SERIAL PRIMARY KEY,
                        temperature_humide FLOAT
                    );
                """)
                
                cur.execute(f"""
                    CREATE TABLE IF NOT EXISTS dim_location (
                        id SERIAL PRIMARY KEY,
                        pays VARCHAR(255),
                        ville VARCHAR(255)
                    );
                """)
                cur.execute(f"""
                    CREATE TABLE IF NOT EXISTS meteo_fait (
                    temps_id INT REFERENCES dim_temps(id),
                    temperature_humide_id INT REFERENCES dim_temperature_humide(id),
                    location_id INT REFERENCES dim_location(id),
                    humidite_relative FLOAT,
                    temperature_air FLOAT
                );
                """)
                conn.commit()
                print("Tables created successfully in database 'meteo_cube'")

    except Exception as e:
        print(f"Error creating database or tables: {e}")
        return False
    
__create_cube_if_not_exist__()

Database 'meteo_cube' already exists
Tables created successfully in database 'meteo_cube'


In [24]:
def __clear_cube__():
    db_name = "meteo_cube"
    try:
        # Connexion à la base de données cible
        with psycopg2.connect(
            dbname=db_name,
            user=POSTGRES_USER,
            password=POSTGRES_PASSWORD,
            host=POSTGRES_HOST
        ) as conn:
            with conn.cursor() as cur:
                # Liste des tables à vider
                tables_to_clear = [
                    "dim_temperature_humide",
                    "dim_temps",
                    "dim_location",
                    "meteo_fait"
                ]
                
                # Exécuter TRUNCATE sur chaque table
                for table in tables_to_clear:
                    cur.execute(f"TRUNCATE TABLE {table} RESTART IDENTITY CASCADE;")
                    print(f"Table '{table}' cleared successfully.")
                
                # Commit des modifications
                conn.commit()

    except Exception as e:
        print(f"Error clearing tables: {e}")
        return False
    
__clear_cube__()

Table 'dim_temperature_humide' cleared successfully.
Table 'dim_temps' cleared successfully.
Table 'dim_location' cleared successfully.
Table 'meteo_fait' cleared successfully.


In [12]:
df_cleaned.head()



,ville,pays,temperature_humide,temperature_air,humidite_specifique,vitesse_vent,intensite_vent,humidite_relative,direction_vent,temperature_point_rosee,composante_est_ouest_vent,pression,precipitations_corrigees,date,heure
0,kedougou,Senegal,9.38,20.57,3.38,-2.29,4.17,22.04,56.7,-1.82,-3.49,98.50,0.0,2025-02-01,00:00:00
1,kedougou,Senegal,8.85,19.44,3.40,-1.85,3.77,23.77,60.7,-1.75,-3.29,98.46,0.0,2025-02-01,01:00:00
2,kedougou,Senegal,8.39,18.40,3.43,-1.45,3.47,25.58,65.3,-1.63,-3.15,98.43,0.0,2025-02-01,02:00:00
3,kedougou,Senegal,8.01,17.36,3.51,-0.79,2.94,27.95,74.4,-1.35,-2.83,98.43,0.0,2025-02-01,03:00:00
4,kedougou,Senegal,7.75,16.34,3.64,0.01,2.44,30.93,90.2,-0.84,-2.44,98.46,0.0,2025-02-01,04:00:00


In [131]:
df_cleaned.columns

Index(['ville', 'pays', 'temperature_humide', 'temperature_air',
       'intensite_vent', 'direction_vent', 'pression', 'vitesse_vent',
       'composante_est_ouest_vent', 'humidite_relative', 'humidite_specifique',
       'temperature_point_rosee', 'precipitations_corrigees', 'date', 'heure'],
      dtype='object')

In [132]:
df_cleaned.dtypes

ville                         object
pays                          object
temperature_humide           float64
temperature_air              float64
intensite_vent               float64
direction_vent               float64
pression                     float64
vitesse_vent                 float64
composante_est_ouest_vent    float64
humidite_relative            float64
humidite_specifique          float64
temperature_point_rosee      float64
precipitations_corrigees     float64
date                          object
heure                         object
dtype: object

In [133]:
df_cleaned.shape

(14616, 15)

In [25]:
import psycopg2
import pandas as pd

def insert_postgres_data(dataframe, batch_size=200):
     # Connexion à PostgreSQL
    conn = psycopg2.connect(dbname=DB_NAME, user=POSTGRES_USER, password=POSTGRES_PASSWORD, host=POSTGRES_HOST)
    cur = conn.cursor()
    
    # Créer la table si elle n'existe pas (en respectant le bon ordre)
    cur.execute(f"""
        CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
            id SERIAL PRIMARY KEY,
            ville VARCHAR(255),
            pays VARCHAR(255),
            temperature_air FLOAT,
            pression FLOAT,
            intensite_vent FLOAT,
            humidite_specifique FLOAT,
            temperature_point_rosee FLOAT,
            composante_est_ouest_vent FLOAT,
            vitesse_vent FLOAT,
            humidite_relative FLOAT,
            direction_vent FLOAT,
            temperature_humide FLOAT,
            precipitations_corrigees FLOAT,
            date DATE,
            heure VARCHAR(255)
        );
    """)
    conn.commit()
    
    # Vérifier le nombre de colonnes dans le DataFrame
    column_count = len(dataframe.columns)
    print(f"DataFrame has {column_count} columns")
    print(f"DataFrame columns: {dataframe.columns.tolist()}")
    
    # Liste des colonnes attendues dans le bon ordre
    expected_columns = [
        'ville', 'pays', 'temperature_air', 'pression', 'intensite_vent', 
        'humidite_specifique', 'temperature_point_rosee', 'composante_est_ouest_vent',
        'vitesse_vent', 'humidite_relative', 'direction_vent', 
        'temperature_humide', 'precipitations_corrigees', 'date', 'heure'
    ]
    
    # Ajuster le DataFrame
    dataframe = dataframe[expected_columns]
        
    # Préparer les données pour l'insertion
    batch = []
    for _, row in dataframe.iterrows():
        row_tuple = tuple(row.values)
        if len(row_tuple) != 15:
            print(f"Warning: Row has {len(row_tuple)} values, expected 15")
            continue
        batch.append(row_tuple)
        
        if len(batch) >= batch_size:
            try:
                query = f"""
                    INSERT INTO {TABLE_NAME} 
                    (
                        ville, pays, temperature_air, pression, intensite_vent, 
                        humidite_specifique, temperature_point_rosee, composante_est_ouest_vent,
                        vitesse_vent, humidite_relative, direction_vent, 
                        temperature_humide, precipitations_corrigees, date, heure
                    )
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                cur.executemany(query, batch)
                conn.commit()
                print(f"Inserted batch of {len(batch)} rows")
                batch = []
            except Exception as e:
                print(f"Error inserting batch: {e}")
                if batch:
                    print(f"First row in batch: {batch[0]}")
                conn.rollback()
    
    # Insérer le dernier batch
    if batch:
        try:
            query = f"""
                INSERT INTO {TABLE_NAME} 
                (
                    ville, pays, temperature_air, pression, intensite_vent, 
                    humidite_specifique, temperature_point_rosee, composante_est_ouest_vent,
                    vitesse_vent, humidite_relative, direction_vent, 
                    temperature_humide, precipitations_corrigees, date, heure
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            cur.executemany(query, batch)
            conn.commit()
            print(f"Inserted final batch of {len(batch)} rows")
        except Exception as e:
            print(f"Error inserting final batch: {e}")
            if batch:
                print(f"First row in batch: {batch[0]}")
            conn.rollback()
    
    print("Data insertion completed")
    cur.close()
    conn.close()


# Exemple d'appel à la fonction
insert_postgres_data(df_cleaned, batch_size=200)


DataFrame has 15 columns
DataFrame columns: ['ville', 'pays', 'intensite_vent', 'humidite_specifique', 'composante_est_ouest_vent', 'temperature_humide', 'pression', 'temperature_air', 'direction_vent', 'humidite_relative', 'vitesse_vent', 'temperature_point_rosee', 'precipitations_corrigees', 'date', 'heure']
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batch of 200 rows
Inserted batc

In [18]:
import psycopg2

def __clear_cube__():
    try:
        # Connexion à la base de données cible
        with psycopg2.connect(
            dbname="meteo_db",
            user="postgres",
            password="passer",
            host="localhost"
        ) as conn:
            with conn.cursor() as cur:
                # Liste des tables à vider
                tables_to_clear = [
                    "meteo_data",

                ]
                
                # Exécuter TRUNCATE sur chaque table
                for table in tables_to_clear:
                    cur.execute(f"TRUNCATE TABLE {table} RESTART IDENTITY CASCADE;")
                    print(f"Table '{table}' cleared successfully.")
                
                # Commit des modifications
                conn.commit()

    except Exception as e:
        print(f"Error clearing tables: {e}")
        return False
    
__clear_cube__()

Table 'meteo_data' cleared successfully.
